In [7]:
import copy_water_052025

In [10]:
protein_pdb = 'protein_pdb_dcd_files/1jwp_wh.pdb'
protein_dcd = 'protein_pdb_dcd_files/1jwp_noaligned_wh.dcd'
output_dir = 'protein_pdb_dcd_files/protein_copied_water'
temp_dir = 'protein_pdb_dcd_files/temp_pdb_dcd'

copy_water_052025.copy_water(protein_pdb, protein_dcd, output_dir, temp_dir, 3205)

Starting trajectory processing...


  0%|          | 0/3205 [00:00<?, ?it/s]

Saved first frame to protein_pdb_dcd_files/protein_copied_water/protein_copied_Water_0.pdb
Processed trajectory saved to protein_pdb_dcd_files/protein_copied_water/protein_copied_water_unaligned.dcd


In [17]:
# Import the custom alignment module
from align_traj import align_protein, align_water_trajectory

# Align the protein-only trajectory:
#  - pdb_path   : input PDB file containing protein + water oxygen (first step output)
#  - dcd_path   : input DCD trajectory (unaligned) for protein + water oxygen
#  - output_pdb : file path where the aligned first-frame PDB will be saved
#  - output_dcd : file path where the aligned DCD trajectory will be saved
align_protein(
    pdb_path='protein_pdb_dcd_files/temp_pdb_dcd/protein_water.pdb',
    dcd_path='protein_pdb_dcd_files/temp_pdb_dcd/protein_water_unaligned.dcd',
    output_pdb='protein_pdb_dcd_files/protein_water_aligned.pdb',
    output_dcd='protein_pdb_dcd_files/protein_water_aligned.dcd'
)

align_water_trajectory(
    reference_pdb='protein_pdb_dcd_files/protein_copied_water/protein_copied_Water_0.pdb',
    trajectory_dcd='protein_pdb_dcd_files/protein_copied_water/protein_copied_water_unaligned.dcd',
    output_directory='protein_water_aligned',
    max_frames=None
)


[Protein Alignment] Started at 2025-05-20 15:01:39
[Protein Alignment] Saved PDB: protein_pdb_dcd_files/protein_water_aligned.pdb
[Protein Alignment] Saved DCD: protein_pdb_dcd_files/protein_water_aligned.dcd
[Protein Alignment] Elapsed time: 11.74 seconds

[Water Alignment] Start at 2025-05-20 15:01:51
[Water Alignment] Saved aligned trajectory: protein_water_aligned/aligned_water_copied_trajectory.dcd
[Water Alignment] Elapsed: 66.96 s



In [21]:
import generate_water_density as gwd
import time
import os

start_time = time.perf_counter()
print(f"GWD Started at {time.strftime('%Y-%m-%d %H:%M:%S')}")

os.makedirs('water_density_maps', exist_ok=True)
    
gwd.generate_water_density_maps(
    pdb_path='protein_pdb_dcd_files/1jwp_wh.pdb',
    dcd_path='protein_pdb_dcd_files/1jwp_noaligned_wh.dcd',
    ref_pdb_path='protein_pdb_dcd_files/protein_copied_water/protein_copied_Water_0.pdb',
    ref_dcd_path='protein_pdb_dcd_files/protein_water_aligned/aligned_water_copied_trajectory.dcd',
    resolution1=2.0,
    resolution2=2.0,
    gpu_id=1
)

elapsed = time.perf_counter() - start_time
print(f"GWD Elapsed time: {elapsed:.2f} seconds\n")

### Note, if you only need the density maps, you can stop at here already! All the density maps have already been generated. 

GWD Started at 2025-05-20 15:26:17


  0%|          | 0/3205 [00:00<?, ?it/s]

  0%|          | 0/3205 [00:00<?, ?it/s]

GWD Elapsed time: 1271.03 seconds



In [1]:
from RMSF_GMM import cluster_high_rmsf_residues

pdb_path       = 'protein_pdb_dcd_files/1jwp_noh.pdb'
dcd_path       = 'protein_pdb_dcd_files/1jwp_noh.dcd'
atom_selection = 'protein and backbone and resid 30 to 280'
cutoff_fraction= 0.25             # 25% of the maximum RMSF
n_components   = 10               # number of GMM clusters
output_csv     = 'cluster_results.csv'
output_plot    = 'rmsf_plot.png'

# Run the RMSF‐clustering workflow:
cluster_high_rmsf_residues(
    pdb_path=pdb_path,
    dcd_path=dcd_path,
    atom_selection=atom_selection,
    cutoff_fraction=cutoff_fraction,
    n_components=n_components,
    output_csv=output_csv,
    output_plot=output_plot
)

## Note, if you want to further run the pocket volume analysis, please select the residues from the clustering results manually. The clustering may not give you the exact results as the protein is highly flexible.

/home/tq19b/.local/lib/python3.9/site-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/home/tq19b/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


Maximum RMSF = 3.901; cutoff = 0.975 (0.25 x max)
RMSF plot saved to rmsf_plot.png


Reading frames: 100%|██████████| 3204/3204 [00:00<00:00, 12382.87it/s]


Cluster results written to cluster_results.csv


In [1]:
from calculate_pocket_volume import analyze_binding_site

pdb_path       = 'protein_pdb_dcd_files/1jwp_noh.pdb'
dcd_path       = 'protein_pdb_dcd_files/1jwp_noh.dcd'
convex_selection='name CA and (resid 236 to 237 or resid 215 to 224 or resid 242 to 245 or resid 268 to 284)'
target_selection='name CA and (resid 236 to 237 or resid 215 to 224 or resid 242 to 245 or resid 268 to 284)'
DENSITY_MAP_DIR='/scratch/tq19b/cdc-wbc-demo/water_density_maps/'
OUTPUT_PREFIX='1jwp'

analyze_binding_site(pdb_path, dcd_path, convex_selection, target_selection, DENSITY_MAP_DIR, OUTPUT_PREFIX)

/home/tq19b/.local/lib/python3.9/site-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/home/tq19b/.local/lib/python3.9/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


Loading trajectory and initializing...
Loading density maps...


Loading density maps: 100%|██████████| 3205/3205 [01:23<00:00, 38.57it/s]


Preparing grid coordinates...
Processing frames...


Frames:   0%|          | 0/3204 [00:00<?, ?it/s]

...

Removing grids too close to protein atoms...
Computing distances between grids and protein atoms...
Filtering grids with density higher than 0.01...
Saving volume grids as PDB file...
Applying Reweighting function...

Processing frame 3202/3204
<AtomGroup [<Atom 1479: CA of type C of resname LYS, resid 215 and segid PROA and altLoc >, <Atom 1488: CA of type C of resname VAL, resid 216 and segid PROA and altLoc >, <Atom 1495: CA of type C of resname ALA, resid 217 and segid PROA and altLoc >, ..., <Atom 1955: CA of type C of resname ILE, resid 282 and segid PROA and altLoc >, <Atom 1963: CA of type C of resname GLY, resid 283 and segid PROA and altLoc >, <Atom 1967: CA of type C of resname ALA, resid 284 and segid PROA and altLoc >]>
Computing sphere...
center: [  7.70977191   5.33697545 -10.51791412]
radius: 10.959028393436288
Filtering grid points inside the sphere...
Generating convex hull using all heavy atoms...
Refining pocket grids inside convex hull...
Removing grids too close t

Frames: 100%|█████████▉| 3203/3204 [08:22<00:00,  6.74it/s]


Processing frame 3203/3204
<AtomGroup [<Atom 1479: CA of type C of resname LYS, resid 215 and segid PROA and altLoc >, <Atom 1488: CA of type C of resname VAL, resid 216 and segid PROA and altLoc >, <Atom 1495: CA of type C of resname ALA, resid 217 and segid PROA and altLoc >, ..., <Atom 1955: CA of type C of resname ILE, resid 282 and segid PROA and altLoc >, <Atom 1963: CA of type C of resname GLY, resid 283 and segid PROA and altLoc >, <Atom 1967: CA of type C of resname ALA, resid 284 and segid PROA and altLoc >]>
Computing sphere...
center: [ 7.99215019  5.08185674 -9.48246492]
radius: 10.281143441854546
Filtering grid points inside the sphere...
Generating convex hull using all heavy atoms...
Refining pocket grids inside convex hull...
Removing grids too close to protein atoms...
Computing distances between grids and protein atoms...
Filtering grids with density higher than 0.01...
Saving volume grids as PDB file...
Applying Reweighting function...

Processing frame 3204/3204
<

Frames: 100%|██████████| 3204/3204 [08:22<00:00,  6.37it/s]


Removing grids too close to protein atoms...
Computing distances between grids and protein atoms...
Filtering grids with density higher than 0.01...
Saving volume grids as PDB file...
Applying Reweighting function...

Processing completed. Saving results...
Plotting results...
All tasks completed successfully.
